# Finetuning models

Fine-tuning ViT16 models with PyTorch Lightning framework. <br/>
End2End model is intended to predict whether nodule on a given scan crop is malignant or benign. Prediction is done soley on the basis of images.<br/>
Second model, Concept model given a scan crop predict typical attributes of nodule (concepts) like: Diameter, Calcification, Subtlety and so on

# Packages

In [1]:
import torch
import numpy as np
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import TQDMProgressBar

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from pathlib import Path

## Modules

In [5]:
%load_ext autoreload
%autoreload 2
from LIDC_Data_Module import LIDC_Data_Module
from ViT_End2End_Model import ViT_End2End_Model_Finetune
from ViT_Concept_Model import ViT_Concept_Model_Finetune 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data_path = "dataset"
weights_path = "weights"
tb_logs_path = "tb_logs"

## Finetuning benigng/malignant End2End ViT model

In [13]:
model_name = "Model_3_ViT"
Path(f"{weights_path}/End2End_ViT/{model_name}").mkdir(parents=True, exist_ok=True)
Path(f"{tb_logs_path}/End2End_ViT/{model_name}").mkdir(parents=True, exist_ok=True)

In [14]:
# training Loop
for fold in range(5): # 5 cross validation folds
    print(f"fold: {fold}")
    trainer = Trainer(
        devices=1,
        accelerator='gpu',
        max_epochs=45,
        log_every_n_steps=22,
        callbacks=[
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(refresh_rate=10)
            ],
        logger=TensorBoardLogger(f"{tb_logs_path}/End2End_ViT/{model_name}", name=f"fold_{fold+1}"),
        precision=16
    )
    
    model = ViT_End2End_Model_Finetune(learning_rate=1e-3, freeze_layers=True, train_layers=40) 
    data_module = LIDC_Data_Module(
        data_dir=data_path,
        fold=fold,
        batch_size=32,
        num_workers=4,
        apply_mask=False,
        finetuning=True
    )
    trainer.fit(model, data_module)
    torch.save(model.state_dict(), f"{weights_path}/End2End_ViT/{model_name}/ViT_finetune_{fold}.pt")
    

/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


fold: 0


Missing logger folder: tb_logs/End2End_ViT/Model_3_ViT/fold_1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | model      | VisionTransformer | 85.8 M
1 | accuracy   | BinaryAccuracy    | 0     
2 | auroc      | BinaryAUROC       | 0     
3 | precision_ | BinaryPrecision   | 0     
4 | recall     | BinaryRecall      | 0     
5 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
21.3 M    Trainable params
64.5 M    Non-trainable params
85.8 M    Total params
343.198   Total estimated model params size (MB)


Params to learn:
	 encoder.layers.encoder_layer_9.ln_1.weight
	 encoder.layers.encoder_layer_9.ln_1.bias
	 encoder.layers.encoder_layer_9.self_attention.in_proj_weight
	 encoder.layers.encoder_layer_9.self_attention.in_proj_bias
	 encoder.layers.encoder_layer_9.self_attention.out_proj.weight
	 encoder.layers.encoder_layer_9.self_attention.out_proj.bias
	 encoder.layers.encoder_layer_9.ln_2.weight
	 encoder.layers.encoder_layer_9.ln_2.bias
	 encoder.layers.encoder_layer_9.mlp.0.weight
	 encoder.layers.encoder_layer_9.mlp.0.bias
	 encoder.layers.encoder_layer_9.mlp.3.weight
	 encoder.layers.encoder_layer_9.mlp.3.bias
	 encoder.layers.encoder_layer_10.ln_1.weight
	 encoder.layers.encoder_layer_10.ln_1.bias
	 encoder.layers.encoder_layer_10.self_attention.in_proj_weight
	 encoder.layers.encoder_layer_10.self_attention.in_proj_bias
	 encoder.layers.encoder_layer_10.self_attention.out_proj.weight
	 encoder.layers.encoder_layer_10.self_attention.out_proj.bias
	 encoder.layers.encoder_layer_10

`Trainer.fit` stopped: `max_epochs=45` reached.


...

fold: 4


Missing logger folder: tb_logs/End2End_ViT/Model_3_ViT/fold_5
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | model      | VisionTransformer | 85.8 M
1 | accuracy   | BinaryAccuracy    | 0     
2 | auroc      | BinaryAUROC       | 0     
3 | precision_ | BinaryPrecision   | 0     
4 | recall     | BinaryRecall      | 0     
5 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
21.3 M    Trainable params
64.5 M    Non-trainable params
85.8 M    Total params
343.198   Total estimated model params size (MB)


Params to learn:
	 encoder.layers.encoder_layer_9.ln_1.weight
	 encoder.layers.encoder_layer_9.ln_1.bias
	 encoder.layers.encoder_layer_9.self_attention.in_proj_weight
	 encoder.layers.encoder_layer_9.self_attention.in_proj_bias
	 encoder.layers.encoder_layer_9.self_attention.out_proj.weight
	 encoder.layers.encoder_layer_9.self_attention.out_proj.bias
	 encoder.layers.encoder_layer_9.ln_2.weight
	 encoder.layers.encoder_layer_9.ln_2.bias
	 encoder.layers.encoder_layer_9.mlp.0.weight
	 encoder.layers.encoder_layer_9.mlp.0.bias
	 encoder.layers.encoder_layer_9.mlp.3.weight
	 encoder.layers.encoder_layer_9.mlp.3.bias
	 encoder.layers.encoder_layer_10.ln_1.weight
	 encoder.layers.encoder_layer_10.ln_1.bias
	 encoder.layers.encoder_layer_10.self_attention.in_proj_weight
	 encoder.layers.encoder_layer_10.self_attention.in_proj_bias
	 encoder.layers.encoder_layer_10.self_attention.out_proj.weight
	 encoder.layers.encoder_layer_10.self_attention.out_proj.bias
	 encoder.layers.encoder_layer_10

`Trainer.fit` stopped: `max_epochs=45` reached.


Epoch 44: 100%|█| 22/22 [00:08<00:00,  2.66it/s, v_num=0, train_loss_step=0.331, train_acc_step=0.833, val_loss=0.381, v


## Finetuning ViT Concept Model

In [31]:
model_name = "Model_3_ViT_concept"
Path(f"{weights_path}/Concept_ViT/{model_name}").mkdir(parents=True, exist_ok=True)
Path(f"{tb_logs_path}/Concept_ViT/{model_name}").mkdir(parents=True, exist_ok=True)

In [32]:
# training Loop
for fold in range(5): # 5 cross validation folds
    print(f"fold: {fold}")
    trainer = Trainer(
        devices=1,
        accelerator='gpu',
        max_epochs=50,
        log_every_n_steps=22,
        callbacks=[
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(refresh_rate=10)
            ],
        logger=TensorBoardLogger(f"{tb_logs_path}/Concept_ViT/{model_name}", name=f"fold_{fold+1}"),
        #precision=16
    )
    
    model_concepts_ViT = ViT_Concept_Model_Finetune(learning_rate=1e-3, train_layers=40, freeze_layers=True)
    data_module = LIDC_Data_Module(
        data_dir=data_path,
        fold=fold,
        labels="concepts",
        batch_size=32,
        num_workers=4,
        apply_mask=False,
        finetuning=True
    )
    trainer.fit(model_concepts_ViT, data_module)
    torch.save(model_concepts_ViT.state_dict(), f"{weights_path}/Concept_ViT/{model_name}/concept_ViT_finetune_{fold}.pt")

/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


fold: 0


Missing logger folder: tb_logs/Concept_ViT/Model_3_ViT_concept/fold_1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | criterion | HuberLoss         | 0     
1 | model     | VisionTransformer | 85.8 M
------------------------------------------------
21.3 M    Trainable params
64.5 M    Non-trainable params
85.8 M    Total params
343.219   Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████████████████████████| 22/22 [00:04<00:00,  4.46it/s, v_num=0, train_loss_step=0.223]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██| 22/22 [00:05<00:00,  4.20it/s, v_num=0, train_loss_step=0.205, val_loss=0.213, train_loss_epoch=0.343]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██| 22/22 [00:05<00:00,  4.21it/s, v_num=0, train_loss_step=0.170, val_loss=0.193, train_loss_epoch=0.211]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██| 22/22 [00:05<00:00,  4.20it/s, v_num=0, train_loss_step=0.193, val_loss=0.177, train_loss_epoch=0.196]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██| 22/22 [00:05<0

`Trainer.fit` stopped: `max_epochs=50` reached.


...

fold: 4


Missing logger folder: tb_logs/Concept_ViT/Model_3_ViT_concept/fold_5
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | criterion | HuberLoss         | 0     
1 | model     | VisionTransformer | 85.8 M
------------------------------------------------
21.3 M    Trainable params
64.5 M    Non-trainable params
85.8 M    Total params
343.219   Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████████████████████████| 22/22 [00:04<00:00,  4.68it/s, v_num=0, train_loss_step=0.178]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██| 22/22 [00:05<00:00,  4.18it/s, v_num=0, train_loss_step=0.252, val_loss=0.198, train_loss_epoch=0.311]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██| 22/22 [00:05<00:00,  4.18it/s, v_num=0, train_loss_step=0.153, val_loss=0.199, train_loss_epoch=0.211]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██| 22/22 [00:05<00:00,  4.16it/s, v_num=0, train_loss_step=0.174, val_loss=0.185, train_loss_epoch=0.203]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██| 22/22 [00:05<0

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█| 22/22 [00:12<00:00,  1.83it/s, v_num=0, train_loss_step=0.158, val_loss=0.133, train_loss_epoch=0.135]


## Evaluating End2End ViT Model

In [28]:
model_name = "Model_1_ViT"

In [29]:
trainer = Trainer(devices=1,
                 accelerator="gpu",
                 precision=16
                 )
acc = []
auroc = []
precision = []
recall = []

for fold in range(5):
    print(f"fold: {fold}")
    data_module = LIDC_Data_Module(data_dir=data_path,
                             fold=fold,
                             batch_size=32,
                             num_workers=4,
                             apply_mask=False,
                             finetuning=True
                            )
    
    model =  ViT_End2End_Model_Finetune()
    model.load_state_dict(torch.load(f"weights/End2End_ViT/{model_name}/ViT_finetune_{fold}.pt"))
    model.eval()
    res = trainer.test(model, data_module)
    acc.append(res[0]["val_acc"])
    auroc.append(res[0]["val_auroc"])
    precision.append(res[0]["val_precision"])
    recall.append(res[0]["val_recall"])

/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


fold: 0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.05it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9239766001701355
        val_auroc           0.9654327630996704
        val_loss            0.2333548218011856
      val_precision         0.9354801177978516
       val_recall           0.9108337163925171
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
fold: 1


/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.54it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc             0.871345043182373
        val_auroc           0.9425939321517944
        val_loss            0.3187585473060608
      val_precision         0.8534777760505676
       val_recall           0.9007518291473389
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
fold: 3


/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.8771929740905762
        val_auroc           0.9585808515548706
        val_loss            0.2726856470108032
      val_precision         0.8923091888427734
       val_recall           0.8728274703025818
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
fold: 4


/home/dzban112/envs/conda_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dzban112/envs/conda_env/lib/python3.11/site-pa ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.8470588326454163
        val_auroc           0.9395918250083923
        val_loss            0.3701825737953186
      val_precision         0.8245460391044617
       val_recall           0.8374471664428711
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


### Average Results of the End2End ViT model:

In [17]:
np.mean(precision)

0.8994231820106506

In [18]:
np.mean(acc)

0.8664465188980103

In [30]:
np.mean(acc)

0.8828620553016663

In [19]:
np.mean(recall)

0.8037772417068482